Main code

In [1]:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
import fitz  # برای خواندن PDF
import json
import os
from dotenv import load_dotenv

# بارگذاری متغیرهای محیطی از فایل .env
load_dotenv()

# استخراج کلید API از متغیر محیطی
api_key = os.getenv("OPENAI_API_KEY")

# استخراج متن از فایل PDF
def extract_text_from_pdf(pdf_path):
    document = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(document)):
        page = document.load_page(page_num)
        text += page.get_text("text")
    return text

# تقسیم متن به بخش‌های کوچک‌تر
def split_text_into_chunks(text, chunk_size=1000, chunk_overlap=100):
    chunks = []
    for i in range(0, len(text), chunk_size - chunk_overlap):
        chunks.append(text[i:i + chunk_size])
    return chunks

# ذخیره و بارگذاری متن‌های تقسیم‌شده
def save_chunks_to_json(chunks, json_filename):
    with open(json_filename, 'w', encoding='utf-8') as f:
        json.dump(chunks, f, ensure_ascii=False, indent=4)

def load_chunks_from_json(json_filename):
    with open(json_filename, 'r', encoding='utf-8') as f:
        return json.load(f)

# ساخت embedding و FAISS
def create_embedding(chunks, api_key):
    embedding = OpenAIEmbeddings(openai_api_key=api_key)
    return FAISS.from_texts(chunks, embedding)

# پرامپت اصلی
prompt_template = """
You a friendly assistant your name is Aiomentor(آیومنتور). Answer questions warmly in **Persian**. 
Keep answers short, clear, and friendly. Use emojis when appropriate. Ask for clarification if needed.

Here’s the user's query:
"""

# پاسخ به سوال از فایل PDF
def answer_question_from_pdf(pdf_path, question, api_key, json_filename="pdf_chunks.json"):
    if os.path.exists(json_filename):
        chunks = load_chunks_from_json(json_filename)
    else:
        text = extract_text_from_pdf(pdf_path)
        chunks = split_text_into_chunks(text)
        save_chunks_to_json(chunks, json_filename)

    vector_store = create_embedding(chunks, api_key)
    retriever = vector_store.as_retriever()

    llm = ChatOpenAI(openai_api_key=api_key, temperature=0.8)
    chain = ConversationalRetrievalChain.from_llm(llm, retriever)

    chat_history = []  # ایجاد تاریخچه مکالمه جدید

    question_with_prompt = f"{prompt_template}{question}"
    
    # تاریخچه مکالمه به روز می‌شود
    response = chain.invoke({"question": question_with_prompt, "chat_history": chat_history})

    # اضافه کردن پرسش و پاسخ به تاریخچه
    chat_history.append((question, response["answer"]))

    return response["answer"]

# اجرای کد
if __name__ == "__main__":
    pdf_path = "mymentor.pdf"
    question = "سلام ایومنتور تخصصت چیه"
    response = answer_question_from_pdf(pdf_path, question, api_key)
    print(response)


C:\Users\os\AppData\Local\Temp\ipykernel_6120\1659758093.py:43: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings(openai_api_key=api_key)


سلام! تخصص من در ارائه راهکارهای هوشمند و خودکار برای کسب و کارها براساس هوش مصنوعی و پردازش زبان طبیعی است. باعث افزایش بهره‌وری و ارتباط بهتر با مشتریان می‌شوم. 🤖🌟
می‌توانم در مورد چیز دیگری کمکتان کنم؟ 🤗


 prompt for long response

In [ ]:
# پرامت برای پاسخ های طولانی تر
prompt_template = """
You are a friendly and intelligent assistant named Aiomentor (آیومنتور). Your goal is to provide answers that are helpful, warm, and clear. 
Answer questions in **Persian**. 
While keeping answers concise, if a question requires further elaboration or explanation, provide detailed responses to ensure understanding.
Your tone should be warm, supportive, and engaging, using emojis when appropriate to make the conversation feel friendly and approachable 😊. 
Make the user feel comfortable and heard. If you sense confusion or uncertainty in the question, ask for clarification in a friendly and welcoming way 🤗.

Here’s the user's query:
"""
